### LazyAdam Optimizer

LazyAdam is a variant of the Adam optimizer that handles sparse updates more efficiently. The original Adam algorithm maintains two moving-average accumulators for each trainable variable; the accumulators are updated at every step. This class provides lazier handling of gradient updates for sparse variables. It only updates moving-average accumulators for sparse variable indices that appear in the current batch, rather than updating the accumulators for all indices. Compared with the original Adam optimizer, it can provide large improvements in model training throughput for some applications. However, it provides slightly different semantics than the original Adam algorithm, and may lead to different empirical results.

In [1]:
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 1.1MB 3.4MB/s 


In [2]:
import tensorflow as tf
import tensorflow_addons as tfa

In [3]:
batch_size = 64
epochs = 10

### Model

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape = (784,), activation = 'relu', name = 'dense_1'),
    tf.keras.layers.Dense(64, activation = 'relu', name = 'dense_2'),
    tf.keras.layers.Dense(10, activation = 'softmax', name = 'predictions'),
])

### Prep Data

In [5]:
dataset = {}
num_validation = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
model.compile(
    optimizer = tfa.optimizers.LazyAdam(0.001), 
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy'])

history = model.fit(
    x_train,
    y_train,
    batch_size = batch_size,
    epochs = epochs)

Epoch 1/10
938/938 [==============================] - 3s 3ms/step - loss: 0.3248 - accuracy: 0.9057
Epoch 2/10
938/938 [==============================] - 3s 3ms/step - loss: 0.1476 - accuracy: 0.9563
Epoch 3/10
938/938 [==============================] - 3s 3ms/step - loss: 0.1082 - accuracy: 0.9670
Epoch 4/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0854 - accuracy: 0.9737
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0683 - accuracy: 0.9789
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0567 - accuracy: 0.9823
Epoch 7/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0466 - accuracy: 0.9857
Epoch 8/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0408 - accuracy: 0.9869
Epoch 9/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0362 - accuracy: 0.9887
Epoch 10/10
938/938 [==============================] - 3s 3ms/step - loss: 0.0307 - accuracy: 0.9901

In [7]:
results = model.evaluate(x_test, y_test, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

79/79 - 0s - loss: 0.0913 - accuracy: 0.9747
Test loss = 0.09132926911115646, Test acc: 0.9746999740600586
